# YouTube Statistic ETL

####  This notebook takes the raw CSV files, converts them into dataframe, transforms it (cleans it) and then uploads them into the PostgreSQL database

In [1]:
# Importing the required modules

import pandas as pd
from sqlalchemy import create_engine, inspect
from credentials import my_password, my_postgres

### Extract CSVs into DataFrames

* Read the two csv files (video-stats.csv, comments.csv) into dataframe using pandas

In [2]:
# Extracting video stats csv file into dataframe

video_stats_file = "Resources/videos-stats.csv"
video_stats_df = pd.read_csv(video_stats_file, index_col=0)
video_stats_df.head()

,Title,Video ID,Published At,Keyword,Likes,Comments,Views
0,Apple Pay Is Killing the Physical Wallet After...,wAZZ-UWGVHI,2022-08-23,tech,3407.0,672.0,135612.0
1,The most EXPENSIVE thing I own.,b3x28s61q3c,2022-08-24,tech,76779.0,4306.0,1758063.0
2,My New House Gaming Setup is SICK!,4mgePWWCAmA,2022-08-23,tech,63825.0,3338.0,1564007.0
3,Petrol Vs Liquid Nitrogen | Freezing Experimen...,kXiYSI7H2b0,2022-08-23,tech,71566.0,1426.0,922918.0
4,Best Back to School Tech 2022!,ErMwWXQxHp0,2022-08-08,tech,96513.0,5155.0,1855644.0


In [3]:
# Extracting comments csv file into dataframe

comments_file = "Resources/comments.csv"
comments_df = pd.read_csv(comments_file, index_col=0)
comments_df.head()

,Video ID,Comment,Likes,Sentiment
0,wAZZ-UWGVHI,Let's not forget that Apple Pay in 2014 requir...,95.0,1.0
1,wAZZ-UWGVHI,Here in NZ 50% of retailers don’t even have co...,19.0,0.0
2,wAZZ-UWGVHI,I will forever acknowledge this channel with t...,161.0,2.0
3,wAZZ-UWGVHI,Whenever I go to a place that doesn’t take App...,8.0,0.0
4,wAZZ-UWGVHI,"Apple Pay is so convenient, secure, and easy t...",34.0,2.0


### Cleaning the data

* Renamed the columns so that it matches with the database
* Filled all the NA with 0's for missing data
* Converted float data in columns 'likes','comments_counts', 'views' to integer
* Converted the date object to pandas date format so it can be loaded into the database date column without any issues
* Dropped duplicate video_id's in order to not voilate primary key constraints in database
* Set video_id as index


In [4]:
# Rename the column headers
video_stats_transformed_df = video_stats_df.rename(columns={"Title":"title",
                                               "Video ID":"video_id",
                                               "Published At":"published_at",
                                               "Keyword":"keywords",
                                               "Likes":"likes",
                                               "Comments":"comments_counts",
                                               "Views":"views"}).copy()

# Filling all the NA with 0
video_stats_transformed_df = video_stats_transformed_df.fillna(0)

# Changing float data types to integer
video_stats_transformed_df[['likes','comments_counts', 'views']] = video_stats_transformed_df[['likes','comments_counts','views']].astype(int)

# Converting date object to date time format
video_stats_transformed_df['published_at'] = pd.to_datetime(video_stats_transformed_df['published_at'])

# Setting the index
video_stats_transformed_df.drop_duplicates("video_id", inplace=True)
video_stats_transformed_df.set_index('video_id', inplace=True)

video_stats_transformed_df.head()

,title,published_at,keywords,likes,comments_counts,views
video_id,,,,,,
wAZZ-UWGVHI,Apple Pay Is Killing the Physical Wallet After...,2022-08-23,tech,3407,672,135612
b3x28s61q3c,The most EXPENSIVE thing I own.,2022-08-24,tech,76779,4306,1758063
4mgePWWCAmA,My New House Gaming Setup is SICK!,2022-08-23,tech,63825,3338,1564007
kXiYSI7H2b0,Petrol Vs Liquid Nitrogen | Freezing Experimen...,2022-08-23,tech,71566,1426,922918
ErMwWXQxHp0,Best Back to School Tech 2022!,2022-08-08,tech,96513,5155,1855644


In [5]:
# Renaming the column headers
comments_transfromed_df = comments_df.rename(columns={"Video ID":"video_id",
                                         "Comment":"comment",
                                         "Likes":"likes",
                                         "Sentiment":"sentiment"}).copy()
# Filling all the NA with 0
comments_transfromed_df = comments_transfromed_df.fillna(0)

# Changing float data types to integer
comments_transfromed_df[['likes','sentiment']] = comments_transfromed_df[['likes','sentiment']].astype(int)
comments_transfromed_df.head()

,video_id,comment,likes,sentiment
0,wAZZ-UWGVHI,Let's not forget that Apple Pay in 2014 requir...,95,1
1,wAZZ-UWGVHI,Here in NZ 50% of retailers don’t even have co...,19,0
2,wAZZ-UWGVHI,I will forever acknowledge this channel with t...,161,2
3,wAZZ-UWGVHI,Whenever I go to a place that doesn’t take App...,8,0
4,wAZZ-UWGVHI,"Apple Pay is so convenient, secure, and easy t...",34,2


### Connecting to PostgreSQL Database

* Connected to youtube_stats_db 
* Created engine using connection string

In [6]:
protocol = 'postgresql'
username = my_postgres
password = my_password
host = 'localhost'
port = 5432
database_name = 'youtube_stats_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

### Load Data to Database

* Checked for existing tables in the database
* Uploaded transformed dataframes into database tables using pandas and sqlalchemy

In [7]:
# Confirming tables

inspector = inspect(engine)
inspector.get_table_names()

['comments', 'video_stats']

In [8]:
# Using pandas to upload transformed video stats dataframe into database

video_stats_transformed_df.to_sql(name='video_stats', con=engine, if_exists='append', index=True)

869

In [9]:
# Using pandas to upload transformed comments dataframe into database

comments_transfromed_df.to_sql(name='comments', con=engine, if_exists='append', index=False)

409